In [1]:
import sys, os
from pathlib import Path
sys.path.append(os.path.abspath(Path().resolve().parent/'src'))


from kinematicsrobotics.model import Cluster
from kinematicsrobotics.datahandler import Save, Extract

In [2]:
ext = Extract()
ext._path_project = os.path.abspath(Path().resolve().parent)

dataset = ext.dataframe(r'src\data\ready\dataset-radius-1cm.csv')
dataset.shape

(6889, 11)

In [3]:
kmeans = Cluster(data = dataset[['p_x','p_y','p_z']], 
                n_clusters = 6, 
                n_init = 'auto'
)

In [14]:
from kinematicsrobotics.model import Model
from kinematicsrobotics.model import Cluster
from kinematicsrobotics.datahandler import Save, Extract

class LocalModel:
    def __init__(self, *, model_cluster: Cluster, **kw):
        self._model_cluster = model_cluster
        self.models(**kw)
    
    
    def models(self, **kw):
        models = []
        for k in self._model_cluster._class_cluster:
            models.append(Model.mlp_regressor(**kw))
        
        self._models = models
    
    def set_model(self,**params):
        for model in self._models:
            model.set_model(**params)

    def fit(self,*,x,y):
        i = 0
        for model in self._models:
            self._models[i] = model.fit(x = x, y = y)
            i +=1

    # def predict(self,*, x):
    #     return self._model.predict(x)

In [7]:
mlps = LocalModel(model_cluster=kmeans,
                early_stopping=True, 
                EPOCHS=1000, 
                EPOCHS_NOCHANGE=5,
                random_state=42, 
                verbose=True)

In [12]:
mlps._models[1]._model

MLPRegressor(early_stopping=True, max_iter=1000, n_iter_no_change=5,
             random_state=42, verbose=True)